In [1]:
import numpy as np

from sim.gen_matrix import *
from scipy.sparse import csr_matrix, csgraph

from sim.world import World

In [ ]:
A = weighted_directed_grid4x4(seed=42)
G = nx.from_scipy_sparse_array(A, create_using=nx.DiGraph, edge_attribute="weight")
layout = nx.layout.spring_layout(G)
ipx.network(G, layout=layout)

In [ ]:
csgraph.breadth_first_order(A, 0, directed=True)[0]

In [2]:
world = World()
bfo = csgraph.breadth_first_order(world.get_map(), 0, directed=True)[0]
bfo

[2026-02-15 16:26:07.038245-05:00] Initializing world...
[2026-02-15 16:26:07.040156-05:00] Parking lots: [4, 2, 13]


array([ 0,  1,  4,  2,  5,  8,  3,  6,  9, 12,  7, 10, 13, 11, 14, 15],
      dtype=int32)

In [5]:
world.get_cost_for_lot(13)

np.int32(2)

In [ ]:
bfo = csgraph.breadth_first_order(world.get_map(), 0, directed=True)[0]
wdm = csgraph.floyd_warshall(world.get_map(), directed=True)
dist_mat = csgraph.floyd_warshall(world.get_map(), directed=True, unweighted=True)
l = [i for i in bfo if world.is_parking_lot(i) and dist_mat[0, i] <= 3]
print(l)

distances = np.array([(ii, wdm[0, ii]) for ii in l])
print(distances)
min_distance = np.argsort(distances, axis=0)
print(min_distance[0, 1])

In [ ]:
def shortest_path_csr(adj: csr_matrix, start: int, end: int):
    # Run Dijkstra from the start node
    dist, predecessors = csgraph.dijkstra(csgraph=adj, directed=True, indices=start, return_predecessors=True)

    distance = dist[end]

    # If unreachable
    if np.isinf(distance):
        return np.inf, []

    # Reconstruct path by walking predecessors backwards
    path = []
    current = end

    while current != -9999:  # SciPy sentinel for no predecessor
        path.append(current)
        if current == start:
            break
        current = predecessors[current]

    path.reverse()
    return distance, path

In [ ]:
dist, path = shortest_path_csr(world.get_map(), 0, 15)
print(dist, path)